# EDA 1 - Exploração Simples

Nessa rápida exploração de dados, o objetivo é obter rápidas informações sobre a Coletânea de Hinos da Igreja Cristã Maranata (ICM) - Edição 2018.

Como descrito na própria coletânea:
> Os hinos e louvores, editados nesta Coletânea, englobam partes de outras edições já publicadas que expressam gratidão, adoração e louvor ao nosso Deus, entoados em todas épocas vividas pela Igreja Fiel, momento também comemorado pela Igreja Cristã Maranata em seus cinquenta anos de existência. 

## Origem dos dados

A extração de dados para construção da base tem por origem slides oficiais disponibilizados pelo Departamento de Louvor da ICM. Todo processo de extração está descrito no notebook `pptx_txt_json.ipynb`, neste mesmo repositório.

Até o momento desta análise, os dados estão salvos em uma base de dados SQLite.



In [ ]:
from sqlalchemy import create_engine

# Create an engine to the SQLite database
engine = create_engine("sqlite:///../assets/database.db")

# Connect to the database
connection = engine.connect()
connection

In [ ]:
import pandas as pd

sql_query = """
select
    numero,
    nome,
    texto,
    texto_limpo,
    categoria_id,
    c.descricao as categoria
from 
    hino
    left join categoria c on c.id = categoria_id
where
    coletanea_id = 1
"""

hinos_analise = pd.read_sql_query(sql_query, connection)
hinos_analise

In [ ]:
hinos_analise.loc[hinos_analise["numero"] == "null", "numero"] = 0
hinos_analise["numero_int"] = hinos_analise["numero"].astype(int)
hinos_analise = (
    hinos_analise.drop(columns=["numero"])
    .rename(columns={"numero_int": "numero"})
    .sort_values("numero")
)
hinos_analise

In [ ]:
from pathlib import Path

assets_folder = Path("../assets")

# save to json and pickle
hinos_analise[["numero", "nome", "texto", "categoria"]].to_json(
    assets_folder / "hinos_icm.json",
    orient="records",
    force_ascii=False,
    indent=4,
    index=False,
)
hinos_analise.to_pickle(assets_folder / "hinos_analise.pkl")

795 louvores ao todo.

## Categorias

In [ ]:
categorias_count = (
    hinos_analise[["categoria_id", "categoria", "numero"]]
    .groupby(["categoria_id", "categoria"])
    .count()
    .reset_index()
    .rename(columns={"numero": "contagem"})
)
categorias_count["categoria_abr"] = categorias_count["categoria"].apply(
    lambda x: x[:13] + "..." if len(x) > 15 else x
)
categorias_count

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="darkgrid")


f, axs = plt.subplots(1, 1, figsize=(12, 8))
sns.barplot(x="contagem", y="categoria", data=categorias_count, ax=axs)
""" sns.barplot(
    x="contagem",
    y="categoria_abr",
    data=categorias_count.sort_values(by="contagem", ascending=False),
    ax=axs[1],
) """